## Importing Essential Libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

In [ ]:
#Reading CSV
df1 = pd.read_csv("../input/bengaluru-house-price-data/datasets_20710_26737_Bengaluru_House_Data.csv")
df1

In [ ]:
#checking dimensions of dataframe
df1.shape

In [ ]:
#checking columns name
df1.columns

In [ ]:
#checking unique values in area_type
df1['area_type'].unique()

In [ ]:
#checking values_count in area_type
df1['area_type'].value_counts()

In [ ]:
#drop area_type society balcony availability because we are building a simple model so find those columns unecessary 
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.shape

In [ ]:
#Checking for null values in dataset
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
#Dropping null value tuples
df3 = df2.dropna()

df3.isnull().sum()

In [ ]:
df3.shape

In [ ]:
#tokenizing Size column and extracting the integer from it and storing it to bhk column
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.bhk.unique()

In [ ]:
#Checking if float
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
#return all rows which are not float
df3[~df3['total_sqft'].apply(is_float)].head(10)

In [ ]:
# function used for tokenizing using - and taking average to scale it properly
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
#tokenizing using - and taking average to scale it properly
df4 = df3.copy()
df4.total_sqft = df4.total_sqft.apply(convert_sqft_to_num)
df4 = df4[df4.total_sqft.notnull()]
df4.head(10)

In [ ]:
#checking that is aove applying successful
df4.loc[30]

In [ ]:
#calculating price price_per_sqft price/total_sqft 
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()

In [ ]:
#calculating price price_per_sqft
df5_stats = df5['price_per_sqft'].describe()
df5_stats

In [ ]:
df5.to_csv("bhp.csv",index=False)

In [ ]:
df5['bhk'].value_counts() 

In [ ]:
# matplotlib.rcParams.update({'font.size':15})
#graph on approx price on basis of bhk
dt=df5
df_filtered = dt[dt['bhk'] <=5]
df_filtered['bhk'].value_counts()
# df_filtered
a1=df5[df5['bhk'] ==1]['price'].mean()
a2=df5[df5['bhk'] ==2]['price'].mean()
a3=df5[df5['bhk'] ==3]['price'].mean()
a4=df5[df5['bhk'] ==4]['price'].mean()
a5=df5[df5['bhk'] ==5]['price'].mean()
labels=['1_BHK','2_BHK','3_BHK','4_BHK','5_BHK']
price=[a1,a2,a3,a4,a5]
plt.bar(labels,price,color='purple')
plt.xlabel('BHK')
plt.ylabel('Average Price (LAKHS)')
plt.title('Approx Price on basis of BHK')
plt.savefig("BHK Vs Price.png")
plt.show()
a4

In [ ]:
# # df_filtered[df_filtered['location'].value_counts()>143]
# df_fil=df_filtered.groupby('location').filter(lambda x : len(x)>143)
# df_fil['location'].value_counts()

In [ ]:
# df_filtered[df_filtered['location'].value_counts()>143]
# Top 10 costly Area_Pricewise 
d00=df5.sort_values('price',ascending=False)
d01=d00['location'].head(10)
d02=d00['price'].head(10)
plt.bar(d01,d02,color='purple')
plt.xlabel('AREA')
plt.ylabel('Price (LAKHS)')
plt.title('Top 10 Costly Area_PriceWise')
plt.grid(True)
plt.savefig("Top 10 Costly Area_PriceWise.png")
plt.show()

In [ ]:
# Demand of Bhk in particulare area
labels=['1_BHK','2_BHK','3_BHK','4_BHK','5_BHK']
bhk_no=[640,5517,4818,1390,349]
plt.pie(bhk_no,labels=labels,radius=1.1,autopct='%0.2f%%')
plt.title('Demand of BHK')
plt.savefig('Pie_Chart_BHK.jpg')
plt.show()

### Renaming all Location which have less than 10 entries in datasets to others 

In [ ]:
df5.location = df5.location.apply(lambda x: x.strip())
location_stats = df5['location'].value_counts(ascending=False)
location_stats

In [ ]:
location_stats.values.sum()

In [ ]:
len(location_stats[location_stats>10])

In [ ]:

len(location_stats)


In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10


In [ ]:
len(df5.location.unique())

In [ ]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

In [ ]:
df5.head(10)

### Removing Outliar of total_sqft in which bhk size is less than 300

In [ ]:
df5[df5.total_sqft/df5.bhk<300].head()

In [ ]:
df5.shape

In [ ]:

df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

### Removing PricePerSqft Outliar According to standard deviation

In [ ]:
df6.price_per_sqft.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape

In [ ]:

def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.bhk==2)]
    bhk3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df7,"Rajaji Nagar")

In [ ]:
plot_scatter_chart(df7,"Hebbal")

In [ ]:
# Removing PricePerSqft Outliar According to standard deviation
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
# df8 = df7.copy()
df8.shape

In [ ]:
plot_scatter_chart(df8,"Rajaji Nagar")


In [ ]:
plot_scatter_chart(df8,"Hebbal")

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")


In [ ]:
df8.bath.unique()


In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

### Removing entries having more than 10 bathrooms

In [ ]:
df8[df8.bath>10]

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df9.head(2)

In [ ]:

df10 = df9.drop(['size','price_per_sqft'],axis='columns')
df10.head(3)


In [ ]:
#One_Hot_Encoding
dummies = pd.get_dummies(df10.location)
dummies.head(3)

In [ ]:
df11 = pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head()
df11.dtypes

In [ ]:
df12 = df11.drop('location',axis='columns')
df12.head(2)

In [ ]:
df12.shape

In [ ]:
X = df12.drop(['price'],axis='columns')
X.head(3)

In [ ]:
X.shape

In [ ]:
y = df12.price
y.head(3)


In [ ]:
len(y)

In [ ]:
# Splitting for training and testing(20% testing)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)



# Building regression model

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

# Performing Cross validation

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

# Comparing Scores using gridsearch cv for 5 algo

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
         'xgboost_regression' : {
            'model': XGBRegressor(),
            'params': {
                
            'max_depth': [4],
            'n_estimators':[500],
            'learning_rate': [0.4]
            }
        },
        'random_forest' : {
            'model': RandomForestRegressor(),
            'params': {               
            'max_depth': [9],
            'n_estimators':[70],        
            'random_state':[42]
            }
        },
#         'Gradient_Boost' : {
#             'model': GradientBoostingRegressor(),
#             'params': {               
#             'max_depth': [9],
#             'n_estimators':[700],        
#             'random_state':[42]
#             }
#         },
           
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

df=find_best_model_using_gridsearchcv(X,y)
df

In [ ]:
# df=find_best_model_using_gridsearchcv(X,y)
# df
#Accuracy Comparison graph
plt.bar(df['model'],df['best_score'],color="orange")
plt.show()

In [ ]:
#function to predict
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

# Testing with some data

In [ ]:
predict_price('1st Phase JP Nagar',1000, 2, 2)

In [ ]:
predict_price('1st Phase JP Nagar',1000, 3, 3)


In [ ]:
predict_price('Indira Nagar',1000, 2, 2)

In [ ]:
predict_price('Indira Nagar',1000, 3, 3)